In [1]:
import math
import subprocess
import pandas as pd
import signal
import re
import scipy.constants as const
import numpy as np

In [2]:
def to_fortran_double(number):
    if number == 0:
        return "0.000d0"
    abs_num = abs(number)
    sign = '-' if number < 0 else ""

    exponent = math.floor(math.log10(abs_num))
    mantissa = abs_num / (10 ** exponent)

    return f"{sign}{mantissa:.3f}d{exponent}"

In [3]:
def update_forward(file_path, fortran_doubles):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()

        # 파라미터 라인을 찾아 수정
        pattern = re.compile(r'\$ double precision, parameter :: (r\d+)= .*')
        for i, line in enumerate(lines):
            match = pattern.match(line)
            if match:
                param_name = match.group(1)
                param_number = int(param_name[1:])
                if param_number < len(fortran_doubles):
                    new_value = fortran_doubles[param_number]
                    lines[i] = f'$ double precision, parameter :: {param_name}= {new_value}\n'

        # 수정된 내용을 파일에 다시 쓰기
        with open('kinet.inp', 'w') as file:
            file.writelines(lines)

        #print(f"{file_path} 파일의 파라미터가 성공적으로 업데이트되었습니다.")

    except FileNotFoundError:
        print(f"오류: {file_path} 파일을 찾을 수 없습니다.")
    except IOError as e:
        print(f"파일 처리 중 오류가 발생했습니다: {e}")
    except Exception as e:
        print(f"예상치 못한 오류가 발생했습니다: {e}")

In [4]:
def update_backward(file_path, fortran_doubles):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()

        # 파라미터 라인을 찾아 수정
        pattern = re.compile(r'\$ double precision, parameter :: (p\d+)= .*')
        for i, line in enumerate(lines):
            match = pattern.match(line)
            if match:
                param_name = match.group(1)
                param_number = int(param_name[1:])
                if param_number < len(fortran_doubles):
                    new_value = fortran_doubles[param_number]
                    lines[i] = f'$ double precision, parameter :: {param_name}= {new_value}\n'

        # 수정된 내용을 파일에 다시 쓰기
        with open('kinet.inp', 'w') as file:
            file.writelines(lines)

        #print(f"{file_path} 파일의 파라미터가 성공적으로 업데이트되었습니다.")

    except FileNotFoundError:
        print(f"오류: {file_path} 파일을 찾을 수 없습니다.")
    except IOError as e:
        print(f"파일 처리 중 오류가 발생했습니다: {e}")
    except Exception as e:
        print(f"예상치 못한 오류가 발생했습니다: {e}")

In [5]:
def run_preprocessor(exe_path):
    try:
        process = subprocess.Popen(
            exe_path,
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL,
            stdin=subprocess.PIPE,
            universal_newlines=True
        )

        while process.poll() is None:
            process.stdin.write('.\n')
            process.stdin.flush()

    except:
        pass

In [6]:
def compile_fortran(name):
    compile_command = [
        "gfortran",
        "-o", name,
        "dvode_f90_m.F90",
        "zdplaskin_m.F90",
        "run_plasRxn_v2.F90",
        "bolsig_x86_64_g.dll"
    ]

    try:
        # 컴파일 명령어 실행
        result = subprocess.run(compile_command, check=True, capture_output=True, text=True)
    except subprocess.CalledProcessError as e:
        # 컴파일 중 오류가 발생하면 오류 메시지 출력
        print("컴파일 실패!")
        print("오류 메시지:")
        print(e.stderr)
    except FileNotFoundError:
        # gfortran이 설치되어 있지 않거나 PATH에 없는 경우
        print("gfortran을 찾을 수 없습니다. gfortran이 설치되어 있고 시스템 PATH에 추가되어 있는지 확인하세요.")

In [7]:
def run_executable(exe_path):
    try:
        # 실행 파일 실행
        process = subprocess.Popen(
            exe_path,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=True,
            bufsize=1
        )

        # 실시간으로 출력 읽기
        while True:
            output = process.stdout.readline()
            if "ZDPlasKin INFO: the density of species not configured for BOLSIG+ solver exceeds 1.00D+00" in output:
                print("지정된 문자열이 감지되었습니다. 프로그램을 종료합니다.")
                process.send_signal(signal.SIGTERM)
                break
            if "PRESS ENTER TO EXIT" in output:
                print("지정된 문자열이 감지되었습니다. 프로그램을 종료합니다.")
                process.send_signal(signal.SIGTERM)
                break
            # if output:
            #     print(output.strip())

        # 프로세스 종료 대기
        return_code = process.wait()

    except:
        pass

In [8]:
forward = np.ones(19)
reverse = np.ones(19)
index = 0
for i in range(10000):
    # zdplaskin.F90 생성
    exe_processor = r'preprocessor.exe'
    run_preprocessor(exe_processor)

    # 실행파일 생성
    exe_path="run_plasRxn_v3.exe"
    compile_fortran(exe_path)

    # 시뮬레이션 실행
    run_executable(exe_path)

    # Read raw data
    conditions = []
    with open('qt_conditions_list.txt','r') as file:
        for line in file:        
            line = line.strip()
            line = line[2:]
            conditions.append(line)
        file.close()

    species = []
    with open('qt_species_list.txt','r') as file:
        i = 0
        for line in file:        
            line = line.strip()
            if i < 9:
                line = line[2:]
            else:
                line = line[3:]
            species.append(line)
            i += 1
        file.close()

    reactions = []
    with open('qt_reactions_list.txt','r') as file:
        for line in file:        
            line = line.strip()
            line = line[2:]
            reactions.append(line)
        file.close()

    df_cd = pd.read_csv('qt_conditions.txt', sep=r'\s+', header=0, names=['Time [s]']+conditions)
    df_sp = pd.read_csv('qt_densities.txt', sep=r'\s+', header=0, names=['Time [s]']+species)
    df_rx = pd.read_csv('qt_rates.txt', sep=r'\s+', header=0, names=['Time [s]']+reactions)

    CH4 = (df_sp['CH4'] + df_sp['CH4(V13)'] + df_sp['CH4(V24)'])/const.N_A*16
    C2H6 = (df_sp['C2H6'] + df_sp['C2H6(V13)'] + df_sp['C2H6(V24)'])/const.N_A*30
    C2H4 = (df_sp['C2H4'] + df_sp['C2H4(V1)'] + df_sp['C2H4(V2)'])/const.N_A*28
    C2H2 = (df_sp['C2H2'] + df_sp['C2H2(V13)'] + df_sp['C2H2(V2)'] + df_sp['C2H2(V5)'])/const.N_A*26
    C3H8 = (df_sp['C3H8'] + df_sp['C3H8(V1)'] + df_sp['C3H8(V2)'])/const.N_A*44
    C3H6 = (df_sp['C3H6'] + df_sp['C3H6(V)'])/const.N_A*42
    C2H = df_sp['C2H']/const.N_A*25
    CH = df_sp['CH']/const.N_A*13
    C3H7 = df_sp['C3H7']/const.N_A*43
    C3H5 = df_sp['C3H5']/const.N_A*41
    CH3 = df_sp['CH3']/const.N_A*15
    CH2 = df_sp['CH2']/const.N_A*14
    C2H3 = df_sp['C2H3']/const.N_A*27
    C4H9 = df_sp['C4H9']/const.N_A*57
    C2H5 = df_sp['C2H5']/const.N_A*29
    C3H4 = df_sp['C3H4']/const.N_A*40
    C5H12 = df_sp['C5H12']/const.N_A*72
    C4H10 = df_sp['C4H9H']/const.N_A*58

    del_CH4 = CH4.iloc[0] - CH4.iloc[-1]
    CH4_conv_cal = del_CH4/CH4.iloc[0] * 100
    total_CH = C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C2H.iloc[-1] + CH.iloc[-1] + C3H7.iloc[-1] + C3H5.iloc[-1] + CH3.iloc[-1] + CH2.iloc[-1] + C2H3.iloc[-1] + C4H9.iloc[-1] + C2H5.iloc[-1] + C3H4.iloc[-1] + C5H12.iloc[-1] + C4H10.iloc[-1]
    C2H6_sel_cal = C2H6.iloc[-1]/total_CH * 100
    C2H4_sel_cal = C2H4.iloc[-1]/total_CH * 100
    C2H2_sel_cal = C2H2.iloc[-1]/total_CH * 100
    C3H8_sel_cal = C3H8.iloc[-1]/total_CH * 100
    C3H6_sel_cal = C3H6.iloc[-1]/total_CH * 100
    C2H_sel_cal = C2H.iloc[-1]/total_CH * 100
    CH_sel_cal = CH.iloc[-1]/total_CH * 100
    C3H7_sel_cal = C3H7.iloc[-1]/total_CH * 100
    C3H5_sel_cal = C3H5.iloc[-1]/total_CH * 100
    CH3_sel_cal = CH3.iloc[-1]/total_CH * 100
    CH2_sel_cal = CH2.iloc[-1]/total_CH * 100
    C2H3_sel_cal = C2H3.iloc[-1]/total_CH * 100
    C4H9_sel_cal = C4H9.iloc[-1]/total_CH * 100
    C2H5_sel_cal = C2H5.iloc[-1]/total_CH * 100
    C3H4_sel_cal = C3H4.iloc[-1]/total_CH * 100
    C5H12_sel_cal = C5H12.iloc[-1]/total_CH * 100
    C4H10_sel_cal = C4H10.iloc[-1]/total_CH * 100    



    # error 계산
    exp = [20, 0, 0, 0, 40, 0, 22, 0, 17, 0, 5, 0, 10, 0, 6, 0, 0]
    cal = [CH4_conv_cal, CH3_sel_cal, CH2_sel_cal, CH_sel_cal, C2H6_sel_cal, C2H5_sel_cal, C2H4_sel_cal,
           C2H3_sel_cal, C2H2_sel_cal, C2H_sel_cal, C3H8_sel_cal, C3H7_sel_cal, C3H6_sel_cal, C3H5_sel_cal,
           C4H10_sel_cal, C4H9_sel_cal, C5H12_sel_cal]
    
    err = 0
    for j in range(len(exp)):
        err += (exp[j] - cal[j])**2
    
    for j in range(len(exp)):
        if exp[j] > cal[j]:
            forward[j] *= 1.1
            reverse[j] *= 0.9
        elif exp[j] < cal[j]:
            forward[j] *= 0.9
            reverse[j] *= 1.1
        else:
            forward[j] *= 1
            reverse[j] *= 1
    
    fortran_forward = list(map(to_fortran_double, forward))
    fortran_reverse = list(map(to_fortran_double, reverse))

    print(f'{index}번째 run 완료! error: {err}')
    index += 1
    file_path = 'kinet.inp'
    update_forward(file_path, fortran_forward)
    update_backward(file_path, fortran_reverse)


    

지정된 문자열이 감지되었습니다. 프로그램을 종료합니다.
0번째 run 완료! error: 3493.7504271829052
지정된 문자열이 감지되었습니다. 프로그램을 종료합니다.
1번째 run 완료! error: 3541.563199601103
지정된 문자열이 감지되었습니다. 프로그램을 종료합니다.
2번째 run 완료! error: 3294.8859814426655
지정된 문자열이 감지되었습니다. 프로그램을 종료합니다.
3번째 run 완료! error: nan


C:\Users\USER\AppData\Local\Temp\ipykernel_17664\116420602.py:72: RuntimeWarning: invalid value encountered in scalar divide
  C2H6_sel_cal = C2H6.iloc[-1]/total_CH * 100
C:\Users\USER\AppData\Local\Temp\ipykernel_17664\116420602.py:73: RuntimeWarning: invalid value encountered in scalar divide
  C2H4_sel_cal = C2H4.iloc[-1]/total_CH * 100
C:\Users\USER\AppData\Local\Temp\ipykernel_17664\116420602.py:74: RuntimeWarning: invalid value encountered in scalar divide
  C2H2_sel_cal = C2H2.iloc[-1]/total_CH * 100
C:\Users\USER\AppData\Local\Temp\ipykernel_17664\116420602.py:75: RuntimeWarning: invalid value encountered in scalar divide
  C3H8_sel_cal = C3H8.iloc[-1]/total_CH * 100
C:\Users\USER\AppData\Local\Temp\ipykernel_17664\116420602.py:76: RuntimeWarning: invalid value encountered in scalar divide
  C3H6_sel_cal = C3H6.iloc[-1]/total_CH * 100
C:\Users\USER\AppData\Local\Temp\ipykernel_17664\116420602.py:77: RuntimeWarning: invalid value encountered in scalar divide
  C2H_sel_cal = C2H.

지정된 문자열이 감지되었습니다. 프로그램을 종료합니다.
4번째 run 완료! error: 3049.94038229486
5번째 run 완료! error: 3713.642961300243
